In [3]:
import tensorflow as tf
import tensorflow.compat.v1 as tf1
import numpy as np
tf1.disable_eager_execution()
tf1.reset_default_graph()
x_data = np.random.rand(100).astype('f')
y_data = x_data * .1 + .3

init = lambda : (tf.Variable(tf.random.uniform([1], -1., 1.), name="Weight"),
                 tf.Variable(tf.random.uniform([1], -1., 1.), name="Bias"))
W, b = init()
yp = lambda x : W*x +b
mse = lambda yp, yt : tf.reduce_mean(tf.square(yp - yt))

way = 2
if way == 1:
    print("Way 1")
    for step in range(10):
        with tf.GradientTape() as tape:
            y = yp(x_data)
            loss = mse(y, y_data)
        grads = tape.gradient(loss, [W, b])
        if step % 200 == 0:
            print(f"\nW : {W.numpy()},\nb = {b.numpy()},\ng[0] : {grads[0].numpy()},\ng[1] :{grads[1].numpy()}")
        W.assign(W - 0.01 * grads[0].numpy())
        b.assign(b - 0.01 * grads[1].numpy())
elif way == 2:
    print("Way 2")
    optimizer = tf.optimizers.SGD(learning_rate=0.1)
    for step in range(10):
        with tf.GradientTape() as tape:
            y = yp(x_data)
            loss = mse(y, y_data)
        grads = tape.gradient(loss, [W, b])
        process_grads = [g for g in grads]
        grads_and_vars = zip(process_grads, [W, b])
        tf1.summary.FileWriter("/tmp/lotto", tf1.get_default_graph())
        if step % 20 == 0:
            print(f"\nW={W}, \nb={b}, \ng={grads[0]}, \ng[1]={grads[1]}")
        optimizer.apply_gradients(grads_and_vars)
elif way == 3:
    print("Way 3")
    y_pred, yt = yp(x_data), y_data
    optimzier = tf.optimizers.SGD(learning_rate=0.1)
    for step in range(201):
        if step % 20 == 0:
            print(f"\nW = {W.numpy()}, \nb = {b.numpy()}")
        mse_min = lambda : tf.reduce_mean(tf.square((W * x_data + b) - yt))
        optimizer.minimize(mse_min, var_list=[W,b])

Way 2

W=<tf.Variable 'Weight:0' shape=(1,) dtype=float32>, 
b=<tf.Variable 'Bias:0' shape=(1,) dtype=float32>, 
g=Tensor("gradient_tape/mul/Reshape:0", shape=(1,), dtype=float32), 
g[1]=Tensor("gradient_tape/add/Reshape:0", shape=(1,), dtype=float32)


# Softmax

In [4]:
import tensorflow as tf
import tensorflow.compat.v1 as tf1
import numpy as np
tf.get_logger().setLevel("ERROR")
# tf1.enable_eager_execution()
tf1.disable_eager_execution()
tf1.reset_default_graph()

xy = np.loadtxt("softmax.csv", unpack=True, delimiter=',', dtype="float32")
x_data = np.transpose(xy[:3])
y_data = np.transpose(xy[3:])
print(f"x_data : {x_data.shape}")
print(f"y_data : {y_data.shape}")

W = tf.Variable(tf.zeros([3, 3]))
hx = lambda X: tf.nn.softmax(tf.matmul(X, W))

cost = lambda Y: tf.reduce_mean(-tf.reduce_sum(Y * tf.math.log(hx(x_data)), 1))
rate = tf.Variable(.01)

opt = tf.optimizers.SGD(rate)
for step in range(2001):
    with tf.GradientTape() as tape:
        L1_reg = 0.1 * tf.reduce_sum(tf.abs(W))
        loss = cost(y_data) + L1_reg
    grads = tape.gradient(loss, [W])
    opt.apply_gradients(zip(grads, [W]))
    
    tf1.summary.FileWriter("/tmp/lotto", tf1.get_default_graph())
    if step % 20 == 0:
        print(f"step:{step}, grads:{grads[0]}, W:{W}") 
    
print('-'*60)
a = hx([[1., 11., 7.]]).numpy()
b = hx([[1., 3., 4.]]).numpy()
c = hx([[1., 1., 0.]]).numpy()
abc = hx([[1., 11, 7], [1., 3., 4.], [1., 1., 0.]])
print(f"[1., 11., 7.] : {a}, {tf.argmax(a, 1).numpy()}")
print(f"[1., 3., 4.] : {b}, {tf.argmax(b, 1).numpy()}")
print(f"[1., 1., 0.] : {c}, {tf.argmax(c, 1).numpy()}")

x_data : (8, 3)
y_data : (8, 3)
step:0, grads:Tensor("AddN:0", shape=(3, 3), dtype=float32), W:<tf.Variable 'Variable:0' shape=(3, 3) dtype=float32>
step:20, grads:Tensor("AddN_20:0", shape=(3, 3), dtype=float32), W:<tf.Variable 'Variable:0' shape=(3, 3) dtype=float32>
step:40, grads:Tensor("AddN_40:0", shape=(3, 3), dtype=float32), W:<tf.Variable 'Variable:0' shape=(3, 3) dtype=float32>


KeyboardInterrupt: 

# XOR TF1.x

In [3]:
import tensorflow as tf
import tensorflow.compat.v1 as tf1
tf.get_logger().setLevel("ERROR")
tf1.disable_eager_execution()
import numpy as np

xy = np.loadtxt("xor.csv", unpack=True, delimiter=',')

x_data = np.transpose(xy[:-1])
y_data = np.reshape(xy[-1], (4, 1))

print(f"x_data : \n{x_data}, \ny_data : \n{y_data}")

X = tf1.placeholder(tf.float32)
Y = tf1.placeholder(tf.float32)

W1 = tf1.Variable(tf.random.uniform([2, 2], -1.0, 1.0))
W2 = tf1.Variable(tf.random.uniform([2, 1], -1.0, 1.0))

b1 = tf1.Variable(tf.zeros([2]))
b2 = tf1.Variable(tf.zeros([1]))

L2 = tf1.sigmoid(tf.matmul(X, W1) + b1)
hypothsis = tf1.sigmoid(tf1.matmul(L2, W2) + b2)

cost = -tf1.reduce_mean(Y * tf1.log(hypothsis) + (1-Y) * tf1.log(1-hypothsis))

rate = tf1.Variable(0.1)
optimizer = tf1.train.GradientDescentOptimizer(rate)
train = optimizer.minimize(cost)

init = tf1.global_variables_initializer()

with tf1.Session() as sess:
    sess.run(init)
    for step in range(10000):
        sess.run(train, feed_dict={X: x_data, Y: y_data})
        if step % 1000 == 999:
            r1, (r2, r3) = sess.run(cost, feed_dict={X: x_data, Y: y_data}), sess.run([W1, W2])
            print(f"step: {step}, {r1}, {np.reshape(r2, (1,4)), np.reshape(r3, (1,2))}")
    print('-'*50)

x_data : 
[[0. 0.]
 [0. 1.]
 [1. 0.]
 [1. 1.]], 
y_data : 
[[0.]
 [1.]
 [1.]
 [0.]]


RuntimeError: tf.placeholder() is not compatible with eager execution.

# XOR TF2.x

In [4]:
import tensorflow as tf
import numpy as np

xy = np.loadtxt("xor.csv", delimiter=',', unpack=True, dtype='f')
x_data = xy[:-1].T
y_data = np.reshape(xy[-1], (4, 1))
dataset = \
tf.data.Dataset.from_tensor_slices((x_data, y_data)).batch(len(x_data))

def preprocess_data(features, labels):
    features = tf.cast(features, tf.float32)
    lables = tf.cast(labels, tf.float32)
    return features, labels

W1 = tf.Variable(tf.random.normal((2, 2)), name="weight1")
b1 = tf.Variable(tf.random.normal((2,)), name="bias1")
W2 = tf.Variable(tf.random.normal((2, 1)), name="weight2")
b2 = tf.Variable(tf.random.normal((1,), name="bias2"))

def neural_net(features):
    layer1 = tf.sigmoid(tf.matmul(features, W1) + b1)
    hypothesis = tf.sigmoid(tf.matmul(layer1, W2) +b2)
    return hypothesis

def loss_fn(hypothesis, labels):
    cost = -tf.reduce_mean(labels * tf.math.log(hypothesis) + (1 - labels) * tf.math.log(1 - hypothesis))
    return cost

optimizer = tf.keras.optimizers.Adam(lr=0.01)

def accuracy_fn(hypothesis, labels):
    predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.float32))
    return accuracy

def grad(hypothesis, features, labels):
    with tf.GradientTape() as tape:
        loss_value = loss_fn(neural_net(features), labels)
    return tape.gradient(loss_value, [W1, W2, b1, b2])

EPOCHS = 5000
for step in range(EPOCHS):
    for features, labels in dataset:
        featuers, labels = preprocess_data(features, labels)
        grads = grad(neural_net(features), features, labels)
        optimizer.apply_gradients(grads_and_vars=zip(grads, [W1, W2, b1, b2]))
        
    if step % 500 == 0:
        print(f"Iter :{step}, loss :{loss_fn(neural_net(features), labels)}")

Iter :0, loss :0.7052617073059082
Iter :500, loss :0.37564024329185486
Iter :1000, loss :0.3546944260597229
Iter :1500, loss :0.3503865599632263
Iter :2000, loss :0.3487494885921478
Iter :2500, loss :0.3479474186897278
Iter :3000, loss :0.347495436668396
Iter :3500, loss :0.34721702337265015
Iter :4000, loss :0.34703508019447327
Iter :4500, loss :0.34691092371940613


In [9]:
correct_prediction = tf.cast(neural_net(features) > .5,
                            dtype = tf.float32)

accuracy_fn(neural_net(features), labels).numpy()
correct_prediction

<tf.Tensor: shape=(4, 1), dtype=float32, numpy=
array([[0.],
       [1.],
       [0.],
       [1.]], dtype=float32)>

# SoftMax 도 Binary 가능

In [24]:
import tensorflow as tf
import tensorflow.compat.v1 as tf1
tf1.disable_eager_execution()
sess = tf1.InteractiveSession()

x_ = [[0, 0], [0, 1], [1, 0], [1, 1]]
expect = [[1, 0], [0, 1], [0, 1], [1, 0]]
x = tf1.placeholder("float", [None, 2])
y_ = tf1.placeholder("float", [None, 2])

num_hidden = 20

W = tf.Variable(tf.random.uniform([2, num_hidden], -.01, .01))
b = tf.Variable(tf.random.uniform([num_hidden], -.01, .01))

hidden = tf.nn.relu(tf.matmul(x, W) + b)

W2 = tf.Variable(tf.random.uniform([num_hidden, 2], -.01, .01))
b2 = tf.Variable(tf.random.uniform([2], -.01, .01))

y = tf.nn.softmax(tf.matmul(hidden, W2) + b2)

cross_entropy = -tf.reduce_sum(y_*tf.math.log(y))
# train_step = tf1.train.GradientDescentOptimizer(.2).minimize(cross_entropy)
train_step = tf1.train.AdamOptimizer(.02).minimize(cross_entropy)

tf1.global_variables_initializer().run()

for step in range(1000):
    feed = {x: x_, y_: expect}
    e, a = sess.run([cross_entropy, train_step], feed)
    if e < 1: break # Early Stopping
    print(f"{step} : {e:.4f}")
print('-'*60)
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
print(f"accruacy: {accuracy.eval(feed)*100:.2f}%")
learned_output = tf.argmax(y, 1)
print(learned_output)

0 : 2.7728
1 : 2.7728
2 : 2.7724
3 : 2.7715
4 : 2.7692
5 : 2.7653
6 : 2.7603
7 : 2.7565
8 : 2.7509
9 : 2.7449
10 : 2.7391
11 : 2.7301
12 : 2.7213
13 : 2.7116
14 : 2.7016
15 : 2.6905
16 : 2.6782
17 : 2.6633
18 : 2.6481
19 : 2.6361
20 : 2.6158
21 : 2.5958
22 : 2.5751
23 : 2.5497
24 : 2.5197
25 : 2.4911
26 : 2.4525
27 : 2.4162
28 : 2.3861
29 : 2.3435
30 : 2.2938
31 : 2.2468
32 : 2.1949
33 : 2.1403
34 : 2.0805
35 : 2.0286
36 : 1.9647
37 : 1.9080
38 : 1.8455
39 : 1.7837
40 : 1.7190
41 : 1.6535
42 : 1.5881
43 : 1.5249
44 : 1.4559
45 : 1.3872
46 : 1.3243
47 : 1.2600
48 : 1.1976
49 : 1.1359
50 : 1.0748
51 : 1.0152
------------------------------------------------------------
accruacy: 100.00%
Tensor("ArgMax_10:0", shape=(None,), dtype=int64)


In [13]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import tensorflow.compat.v1 as tf1
tf1.disable_eager_execution()

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train.reshape(-1, 784), x_test.reshape(-1, 784)
y_train, y_test = to_categorical(y_train, 10), to_categorical(y_test, 10) # one hot encoding 주의
learning_rate = 0.001
training_epochs = 15
batch_size = 100
display_step = 1

X = tf1.placeholder(tf.float32, [None, 784])
Y = tf1.placeholder(tf.float32, [None, 10])

W1 = tf.Variable(tf.random.normal([784, 256]))
B1 = tf.Variable(tf.random.normal([256]))
L1 = tf.nn.relu(tf.add(tf.matmul(X, W1), B1))

W2 = tf.Variable(tf.random.normal([256, 256]))
B2 = tf.Variable(tf.random.normal([256]))
L2 = tf.nn.relu(tf.add(tf.matmul(L1, W2), B2))

W3 = tf.Variable(tf.random.normal([256, 10]))
B3 = tf.Variable(tf.random.normal([10]))
hypothesis = tf.add(tf.matmul(L2, W3), B3)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y))
optimizer = tf1.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

init = tf1.global_variables_initializer()
with tf1.Session() as sess:
    sess.run(init)
    for epoch in range(training_epochs):
        avg_cost = 0.
        training_batch = zip(range(0, len(x_train), batch_size), range(batch_size, len(x_train)+1, batch_size))
        for start, end in training_batch:
            batch_xs, batch_ys = x_train[start:end], y_train[start:end]
            _, c = sess.run([optimizer, cost], feed_dict = {X: batch_xs, Y: batch_ys})
            avg_cost += c / int(len(y_train)/batch_size)
        if (epoch+1) % display_step == 0:
            print(f"Epoch : {epoch+1}, cost : {avg_cost:4f}")
                
    print("Optimization Finished!")
    correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print(f"Accuracy : {accuracy.eval({X:x_test, Y:y_test})*100:.2f}")

Epoch : 1, cost : 43072.507978
Epoch : 2, cost : 10495.201650
Epoch : 3, cost : 6577.473928
Epoch : 4, cost : 4544.423878
Epoch : 5, cost : 3272.067510
Epoch : 6, cost : 2389.454336
Epoch : 7, cost : 1786.702470
Epoch : 8, cost : 1330.298726
Epoch : 9, cost : 980.887135
Epoch : 10, cost : 737.370490
Epoch : 11, cost : 580.988824
Epoch : 12, cost : 436.568086
Epoch : 13, cost : 354.536136
Epoch : 14, cost : 247.551312
Epoch : 15, cost : 213.671548
Optimization Finished!
Accuracy : 94.76
